In [ ]:
from notebook_setup import *

### Global grid

In [ ]:
# Step 1: Create pre-computed grid (run once)
if False:
    grid_5deg = create_global_grid_with_land_classification(
        tile_size_deg=10.0,
        min_land_percentage=1.0,  # minimum land to include tile
        overwrite=False,
        grid_asset_path=GLOBAL_GRID_ASSET_PATH,
    )

### Global Predictions

In [ ]:
run_predictions = True

if run_predictions:
    results_global_optimized = predict_npp_tiled_batch_optimized(
        aoi=None,  # Global (all land tiles)
        tile_size_deg=10.0,
        model_version=GEE_MODEL_VERSION,
        resolution=SPATIAL_RESOLUTION,
        max_tiles_per_batch=3000,
        min_lat=-80,
        max_lat=80,
        model_asset_path_mean=f"{MODEL_ASSETS_PATH}/potential_npp_classifier_fulltraining",
        model_asset_path_std=f"{MODEL_ASSETS_PATH}/potential_npp_std_classifier_fulltraining",
        tiled_folder=NPP_PREDICTION_ASSET_PATH,
        grid_asset_path=GLOBAL_GRID_ASSET_PATH,
    )

else:
    print("Set run_predictions = True to start predictions")

### Calculate Deciles

In [ ]:
tiled_folder_path = f"{NPP_PREDICTION_ASSET_PATH}"
tile_assets = ee.data.listAssets({"parent": tiled_folder_path})
tile_paths = [asset["id"] for asset in tile_assets.get("assets", [])]
prediction_collection = ee.ImageCollection(tile_paths)

# Calculate deciles for Functional Integrity (Z-score)
deciles = calculate_percentiles_from_collection(
    prediction_collection, band_name="functional_integrity", samples_per_tile=10000
).getInfo()

print("Functional Integrity Deciles:", deciles)